# MAP PLOTTER

### *Librerias a utilizar*

In [ ]:
import pandas as pd
import geopandas as gpd
import json
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
pio.renderers.default = "browser"

## *Carga de CSV's*

In [ ]:
# Datos de latitud y longitud + data util (presion, superperficie, radiacion, precipitacion, temperatura).

df = pd.read_csv("C:\\Users\\aabm7\\Downloads\\solar_merged_clean.csv")  # <- Ruta del csv con datos procesados

# Carga de datos de mapa
world_map_data = json.load(open("C:\\Users\\aabm7\\Downloads\\World data\\countries.geo.json")) # <- Ruta del geojson del mundo
panama_map_data = json.load(open("C:\\Users\\aabm7\\Downloads\\Panama_Corregimientos_Boundaries_2024_-9001584211130571386.geojson", encoding="utf-8")) # <- Ruta del geojson de Panamá

# Muestra de datos
print(df.head())


   Cloud_Cover_Mean_24h        date        lon       lat  elevation  \
0              0.839638  2025-01-01 -82.987034  8.348225        210   
1              0.743050  2025-01-02 -82.987034  8.348225        210   
2              0.851710  2025-01-03 -82.987034  8.348225        210   
3              0.693706  2025-01-04 -82.987034  8.348225        210   
4              0.603061  2025-01-05 -82.987034  8.348225        210   

   relative_humidity  surface_net_solar_radiation_sum  surface_pressure  \
0          92.573794                       13327212.0      99120.473796   
1          91.319985                       13731542.0      99211.631185   
2          87.947759                       12416618.0      99237.559733   
3          87.855629                       13795528.0      99244.935710   
4          84.954735                       16216810.0      99202.340007   

   temperature_2m_C  total_precipitation_sum  wind_direction  wind_speed  
0         24.184411                 0.022562   

## Valores promedio de variables 

In [ ]:
# Variables a promediar
vars_mean = [
    'Cloud_Cover_Mean_24h',
    'elevation',
    'relative_humidity',
    'surface_net_solar_radiation_sum',
    'surface_pressure',
    'temperature_2m_C',
    'total_precipitation_sum',
    'wind_direction',
    'wind_speed',
]

## *Merge de mapas*

In [ ]:
# Conversion del dataframe
gdf_points = gpd.GeoDataFrame(
    df, geometry = gpd.points_from_xy(df['lon'], df['lat']),
    crs = "EPSG:4326"
)

# Carga de los limites de corregimientos
gdf_bound = gpd.read_file("C:\\Users\\aabm7\\Downloads\\Panama_Corregimientos_Boundaries_2024_-9001584211130571386.geojson", encoding="utf-8")

# Union espacial de puntos con limites
joined = gpd.sjoin(gdf_points, gdf_bound, how="left", predicate="within")

# Base
base = gdf_bound[["ID_CORR", "Provincia", 'Corregimiento']].copy()

base["ID_CORR"] = base["ID_CORR"].astype(str)
joined["ID_CORR"] = joined["ID_CORR"].astype(str)

# Stats solo donde hay puntos
corr_stats = (
    joined.groupby("ID_CORR")
    .agg({c: "mean" for c in vars_mean})
    .reset_index()
)

# Merge de todos los corregimientos
full = base.merge(corr_stats, on="ID_CORR", how="left")

# Media provincial para rellenar faltantes
prov_means = (
    full.groupby("Provincia")[vars_mean]
    .mean()
    .add_prefix("prov_")
    .reset_index()
)

full = full.merge(prov_means, on="Provincia", how="left")

# Relleno con media provincial
for c in vars_mean:
    global_mean = full[c].mean()
    full[c + "_filled"] = full[c].fillna(full["prov_" + c]).fillna(global_mean)

# Merge final con los datos rellenados
filled_cols = [c + "_filled" for c in vars_mean]
to_merge = ["ID_CORR", "Provincia", "Corregimiento"] + filled_cols

gdf_bound["ID_CORR"] = gdf_bound["ID_CORR"].astype(str)

# Final merge
gdf_bound2 = gdf_bound.merge(full[to_merge], on="ID_CORR", how="left", suffixes=("", "_dup"))

for col in ["Provincia", "Corregimiento"]:
    dup = col + "_dup"
    if dup in gdf_bound2.columns:
        gdf_bound2[col] = gdf_bound2[col].fillna(gdf_bound2[dup])
        gdf_bound2.drop(columns=[dup], inplace=True)


## *PLOTTING*

In [10]:
for ft in panama_map_data.get("features", []):
    props = ft.get("properties", {})
    if "ID_CORR" in props and props["ID_CORR"] is not None:
        props["ID_CORR"] = str(props["ID_CORR"])


fig = px.choropleth_mapbox(
    gdf_bound2,
    geojson=panama_map_data,
    locations="ID_CORR",
    featureidkey="properties.ID_CORR",
    color="surface_net_solar_radiation_sum_filled",
    mapbox_style="open-street-map",
    zoom=6,
    center={"lat": df["lat"].mean(), "lon": df["lon"].mean()},
    opacity=0.75,
    height=700,
    color_continuous_scale=[
        (0.0, "#FFFF00"),  # amarillo
        (0.5, "#FFA500"),  # naranja
        (1.0, "#FF0000"),  # rojo
    ],
    hover_name="Corregimiento",
    hover_data={
        "ID_CORR": False,
        "Provincia": True,

        "Cloud_Cover_Mean_24h_filled": ":.2f",
        "elevation_filled": ":.0f",
        "relative_humidity_filled": ":.2f",
        "surface_net_solar_radiation_sum_filled": ":.2f",
        "surface_pressure_filled": ":.2f",
        "temperature_2m_C_filled": ":.2f",
        "total_precipitation_sum_filled": ":.2f",
        "wind_direction_filled": ":.0f",
        "wind_speed_filled": ":.2f",
    },
)

fig.update_layout(margin=dict(r=0, t=0, l=0, b=0))
fig.show()


C:\Users\aabm7\AppData\Local\Temp\ipykernel_13296\2759260319.py:7: DeprecationWarning:

*choropleth_mapbox* is deprecated! Use *choropleth_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

